In [2]:
import torch
device = torch.device("cuda")
print(device)

cuda


In [3]:
import os
HOME = os.getcwd()
print(HOME)

C:\Users\mjaye\PycharmProjects\SelectAutomation


In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.249  Python-3.11.8 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Setup complete  (16 CPUs, 15.3 GB RAM, 840.9/931.3 GB disk)


In [5]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

In [ ]:
import supervision as sv

# extracts first frame from the video, we can loop over to get every frame.
generator = sv.get_video_frames_generator(MALL_VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_ultralytics(results)
detections = detections[detections.class_id == 0] 
'''class_id = var where var is related to person, in this case 0. Based on our custom data
we can replace [...this...] with our own custom logical statement.'''

# annotate
box_annotator = sv.BoxAnnotator(thickness=4)
labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

%matplotlib inline
try:
    sv.show_frame_in_notebook(frame)
except:
    import cv2
    cv2.imshow("frame", frame)
    cv2.waitKey(0)

In [ ]:
import numpy as np
import supervision as sv

'''first we manually find the co-ordinates of our polygon zone and define it as a numpy array, we 
can do this using segmentation. (learn more)'''
polygon = np.array([
    [1900, 1250],
    [2350, 1250],
    [3500, 2160],
    [1250, 2160]
])
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
zone = sv.PolygonZone(polygon=polygon)

# instantiate annotators
box_annotator = sv.BoxAnnotator(thickness=4)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

# extract video frame
generator = sv.get_video_frames_generator(VIDEO_PATH)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model(frame, imgsz=1280)[0]
detections = sv.Detections.from_ultralytics(results)
detections = detections[detections.class_id == 0]
zone.trigger(detections=detections)

# annotate
box_annotator = sv.BoxAnnotator(thickness=4)
labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
frame = zone.annotator.annotate(scene = frame)

In [ ]:
import numpy as np
import supervision as sv
from ultralytics import YOLO

VIDEO_PATH = "mall.mp4"
HOME = "."
model = YOLO("yolov8s.pt")

polygon = np.array([
    [1725, 1550],
    [2725, 1550],
    [3500, 2160],
    [1250, 2160]
])

video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

zone = sv.PolygonZone(
    polygon=polygon,
    frame_resolution_wh=video_info.resolution_wh
)

box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_scale=1.5, text_thickness=3)

zone_annotator = sv.PolygonZoneAnnotator(
    zone=zone,
    color=sv.Color.white(),
    thickness=4
)

tracker = sv.ByteTrack()

def process_frame(frame: np.ndarray, _) -> np.ndarray:
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[detections.class_id == 0]
    detections = tracker.update_with_detections(detections) # track movement.

    zone.trigger(detections=detections)

    labels = [
        f"person {conf:.2f}"
        for conf in detections.confidence
    ]

    frame = box_annotator.annotate(scene=frame, detections=detections)
    frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(scene=frame)

    return frame

sv.process_video(
    source_path=VIDEO_PATH,
    target_path=f"{HOME}/mall-result.mp4",
    callback=process_frame
)